# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 12:56PM,239573,10,CIF06142022-01,Cosmo International Fragrances,Released
1,Jun 14 2022 12:49PM,239572,19,ACG-2102490323,ACG North America LLC,Released
2,Jun 14 2022 12:35PM,239570,16,8140955,Sartorius Bioprocess Solutions,Released
3,Jun 14 2022 12:31PM,239569,10,SOIVA0000483,Ivaoes Animal Health,Released
4,Jun 14 2022 12:31PM,239569,10,SOIVA0000484,Ivaoes Animal Health,Released
5,Jun 14 2022 12:31PM,239569,10,SOIVA0000485,Ivaoes Animal Health,Released
6,Jun 14 2022 12:31PM,239569,10,SOIVA0000486,Ivaoes Animal Health,Released
7,Jun 14 2022 12:31PM,239569,10,SOIVA0000487,Ivaoes Animal Health,Released
8,Jun 14 2022 12:31PM,239569,10,SOIVA0000488,Ivaoes Animal Health,Released
9,Jun 14 2022 12:31PM,239568,10,SONSB0001819,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,239568,Released,1
46,239569,Released,6
47,239570,Released,1
48,239572,Released,1
49,239573,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239568,NaN,NaN,1.0
239569,NaN,NaN,6.0
239570,NaN,NaN,1.0
239572,NaN,NaN,1.0
239573,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0.0,26.0,128.0
239479,0.0,0.0,33.0
239481,0.0,0.0,1.0
239482,0.0,0.0,1.0
239485,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0,26,128
239479,0,0,33
239481,0,0,1
239482,0,0,1
239485,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,0,26,128
1,239479,0,0,33
2,239481,0,0,1
3,239482,0,0,1
4,239485,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,,26,128
1,239479,,,33
2,239481,,,1
3,239482,,,1
4,239485,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health
9,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC"
10,Jun 14 2022 12:26PM,239567,19,"Zymergen, Inc."
11,Jun 14 2022 12:25PM,239566,19,"GCH Granules USA, Inc."
12,Jun 14 2022 12:23PM,239565,18,"ClearSpec, LLC"
13,Jun 14 2022 12:23PM,239564,18,"ClearSpec, LLC"
14,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances,,,1
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC,,,1
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions,,,1
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health,,,6
4,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC",,,1
5,Jun 14 2022 12:26PM,239567,19,"Zymergen, Inc.",,,1
6,Jun 14 2022 12:25PM,239566,19,"GCH Granules USA, Inc.",,,1
7,Jun 14 2022 12:23PM,239565,18,"ClearSpec, LLC",,,1
8,Jun 14 2022 12:23PM,239564,18,"ClearSpec, LLC",,,1
9,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances,1,,
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC,1,,
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions,1,,
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health,6,,
4,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC",1,,
5,Jun 14 2022 12:26PM,239567,19,"Zymergen, Inc.",1,,
6,Jun 14 2022 12:25PM,239566,19,"GCH Granules USA, Inc.",1,,
7,Jun 14 2022 12:23PM,239565,18,"ClearSpec, LLC",1,,
8,Jun 14 2022 12:23PM,239564,18,"ClearSpec, LLC",1,,
9,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances,1,,
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC,1,,
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions,1,,
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health,6,,
4,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances,1.0,NaN,NaN
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC,1.0,NaN,NaN
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health,6.0,NaN,NaN
4,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:56PM,239573,10,Cosmo International Fragrances,1.0,0.0,0.0
1,Jun 14 2022 12:49PM,239572,19,ACG North America LLC,1.0,0.0,0.0
2,Jun 14 2022 12:35PM,239570,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Jun 14 2022 12:31PM,239569,10,Ivaoes Animal Health,6.0,0.0,0.0
4,Jun 14 2022 12:31PM,239568,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5748922,275.0,28.0,3.0
12,239544,0.0,2.0,0.0
15,958043,179.0,41.0,0.0
16,239570,1.0,0.0,0.0
18,479129,2.0,0.0,0.0
19,1197828,5.0,0.0,0.0
20,239535,20.0,10.0,0.0
21,718448,3.0,0.0,0.0
22,239497,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5748922,275.0,28.0,3.0
1,12,239544,0.0,2.0,0.0
2,15,958043,179.0,41.0,0.0
3,16,239570,1.0,0.0,0.0
4,18,479129,2.0,0.0,0.0
5,19,1197828,5.0,0.0,0.0
6,20,239535,20.0,10.0,0.0
7,21,718448,3.0,0.0,0.0
8,22,239497,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,275.0,28.0,3.0
1,12,0.0,2.0,0.0
2,15,179.0,41.0,0.0
3,16,1.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,5.0,0.0,0.0
6,20,20.0,10.0,0.0
7,21,3.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,275.0
1,12,Released,0.0
2,15,Released,179.0
3,16,Released,1.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,2.0,41.0,0.0,0.0,0.0,10.0,0.0,0.0
Released,275.0,0.0,179.0,1.0,2.0,5.0,20.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,2.0,41.0,0.0,0.0,0.0,10.0,0.0,0.0
2,Released,275.0,0.0,179.0,1.0,2.0,5.0,20.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,2.0,41.0,0.0,0.0,0.0,10.0,0.0,0.0
2,Released,275.0,0.0,179.0,1.0,2.0,5.0,20.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()